In [ ]:
import numpy as np
from MonaiUNets import UNetHeart_8_4_4, UNetHeart_16_4_8
import matplotlib.pyplot as plt
import nibabel as nib
import torch
import os
import onnx
import scipy.ndimage
from skimage.metrics import structural_similarity as ssim

In [ ]:
import nibabel as nib

# List of all patient paths
patient_ids = [
    "patient001_frame01", "patient001_frame12",
    "patient002_frame01", "patient002_frame12",
    "patient003_frame01", "patient003_frame15",
    "patient004_frame01", "patient004_frame15",
    "patient005_frame01", "patient005_frame13",
]

# Dictionary to hold loaded data
all_images = {}
all_labels = {}

for path in patient_ids:
    img_path = f"Task500_ACDC/imagesTr/{path}_0000.nii.gz"
    gt_path = f"Task500_ACDC/labelsTr/{path}.nii.gz"

    img = nib.load(img_path).get_fdata()
    gt = nib.load(gt_path).get_fdata()

    all_images[path] = img
    all_labels[path] = gt

    print(f"✓ Loaded: {path} → image shape = {img.shape}, label shape = {gt.shape}")


In [ ]:
# Transpose all images and labels so that slices are first
for path in patient_ids:
    all_images[path] = all_images[path].transpose(2, 0, 1)  # shape: (slices, H, W)
    all_labels[path] = all_labels[path].transpose(2, 0, 1)

    print(f"✓ Transposed: {path} → new image shape = {all_images[path].shape}, label shape = {all_labels[path].shape}")


In [ ]:
# load pretrained model
model = UNetHeart_8_4_4()

In [ ]:
all_predictions = {}
all_segmentations = {}

for path in patient_ids:
    print(f"\n▶ Performing segmentation for: {path}")

    img = all_images[path]  # already transposed to (slices, H, W)

    # perform segmentation
    pred = model.predict(img)  # shape: (slices, classes, H, W)
    print(np.shape(pred))

    result = np.argmax(pred, axis=1)  # shape: (slices, H, W)
    print(np.shape(result))

    all_predictions[path] = pred
    all_segmentations[path] = result


In [ ]:
for path in patient_ids:
    num_slices = img.shape[0]  
    print(num_slices)

In [ ]:
# --- Görselleştirme (veya kayıt) ---
# Değişkenler zaten bellekte:
#   patient_ids, all_images, all_labels, all_segmentations

for path in patient_ids:
    img     = all_images[path]          # (slices, H, W)
    gt      = all_labels[path]          # (slices, H, W)
    result  = all_segmentations[path]   # (slices, H, W)

    for slide in range(img.shape[0]):
        plt.figure(figsize=(12, 4))

        # Orijinal
        plt.subplot(1, 3, 1)
        plt.imshow(img[slide], cmap="gray")
        plt.title("Original")
        plt.axis("off")

        # Ground truth
        plt.subplot(1, 3, 2)
        plt.imshow(img[slide], cmap="gray")
        plt.imshow(gt[slide], alpha=0.4)
        plt.title("Ground Truth")
        plt.axis("off")

        # Segmentasyon sonucu
        plt.subplot(1, 3, 3)
        plt.imshow(img[slide], cmap="gray")
        plt.imshow(result[slide], alpha=0.4)
        plt.title("Segmentation")
        plt.axis("off")

        plt.suptitle(f"{path} – Slice {slide}", fontsize=10)
        plt.tight_layout()
        plt.show()        # yerine kaydetmek istersen: plt.savefig(...); plt.close()


In [ ]:
for path in patient_ids:
    print(f"\n▶ {path}")
    img = all_images[path]   
    gt  = all_labels[path]   

    mask = np.zeros_like(img)
    indices = np.where(gt == 3)
    mask[indices] = 1

    for slide in range(img.shape[0]):
        plt.imshow(mask[slide])
        plt.title(f"{path} – Slice {slide} (LV Mask)")
        plt.show()

In [ ]:
# adjust images by scaling pixels of the left ventricle
for path in patient_ids:
    print(f"\n▶ {path}")
    img = all_images[path]   
    gt  = all_labels[path]   
    style_adjusted_input = img.copy()
    print(np.shape(indices))
    s = 0.1
    style_adjusted_input[indices] *= s 

In [ ]:
all_predictions = {}
all_segmentations = {}

for path in patient_ids:
    print(f"\n▶ Performing segmentation for: {path}")

    img = all_images[path]  # already transposed to (slices, H, W)

    # perform segmentation
    pred = model.predict(img)  # shape: (slices, classes, H, W)
    print(np.shape(pred))

    result = np.argmax(pred, axis=1)  # shape: (slices, H, W)
    print(np.shape(result))

    all_predictions[path] = pred
    all_segmentations[path] = result
    # --- Görselleştirme (veya kayıt) ---
# Değişkenler zaten bellekte:
#   patient_ids, all_images, all_labels, all_segmentations

for path in patient_ids:
    img     = all_images[path]          # (slices, H, W)
    gt      = all_labels[path]          # (slices, H, W)
    result  = all_segmentations[path]   # (slices, H, W)

    for slide in range(img.shape[0]):
        plt.figure(figsize=(12, 4))

        # Orijinal
        plt.subplot(1, 3, 1)
        plt.imshow(img[slide], cmap="gray")
        plt.title("Original")
        plt.axis("off")

        # Ground truth
        plt.subplot(1, 3, 2)
        plt.imshow(img[slide], cmap="gray")
        plt.imshow(gt[slide], alpha=0.4)
        plt.title("Ground Truth")
        plt.axis("off")

        # Segmentasyon sonucu
        plt.subplot(1, 3, 3)
        plt.imshow(img[slide], cmap="gray")
        plt.imshow(result[slide], alpha=0.4)
        plt.title("Segmentation")
        plt.axis("off")

        plt.suptitle(f"{path} – Slice {slide}", fontsize=10)
        plt.tight_layout()
        plt.show()        # yerine kaydetmek istersen: plt.savefig(...); plt.close()



In [ ]:
# show result
#slide = 5
for path in patient_ids:
    for slide in range(img.shape[0]):
        plt.imshow(style_adjusted_input[slide],cmap="gray")
        plt.title(f"Adjusted for slide {path} and {slide}")
        plt.show()

In [ ]:
dummy_input = torch.from_numpy(img[None, None, 0]).float()  
torch.onnx.export(model.model.model, dummy_input, "model.onnx")


In [ ]:
print(type(model))
print(type(model.model))
print(type(model.model.model))


In [ ]:
for name, module in model.model.model.named_modules():
    print(name, '->', module.__class__.__name__)


In [ ]:

target_layers = [
    "0.conv.unit0.adn.N",
    "0.conv.unit1.adn.N",
    "0.conv.unit2.adn.N",
    "0.conv.unit3.adn.N",
    "1.submodule.0.conv.unit0.adn.N",
    "1.submodule.0.conv.unit1.adn.N",
    "1.submodule.0.conv.unit2.adn.N",
    "1.submodule.0.conv.unit3.adn.N",
    "1.submodule.1.submodule.0.conv.unit0.adn.N",
    "1.submodule.1.submodule.0.conv.unit1.adn.N",
    "1.submodule.1.submodule.0.conv.unit2.adn.N",
    "1.submodule.1.submodule.0.conv.unit3.adn.N",
    "1.submodule.1.submodule.1.submodule.0.conv.unit0.adn.N",
    "1.submodule.1.submodule.1.submodule.0.conv.unit1.adn.N",
    "1.submodule.1.submodule.1.submodule.0.conv.unit2.adn.N",
    "1.submodule.1.submodule.1.submodule.0.conv.unit3.adn.N",
    "1.submodule.1.submodule.2.0.adn.N",
    "1.submodule.1.submodule.2.1.conv.unit0.adn.N",
    "1.submodule.2.0.adn.N",
    "1.submodule.2.1.conv.unit0.adn.N",
    "2.0.adn.N"
]

In [ ]:

for path in patient_ids:
    features_orig_dict  = [dict() for _ in range(10)]
    features_bright_dict = [dict() for _ in range(10)]
    img_bright = style_adjusted_input.copy()

    def get_hook(name, storage_dict):
        def hook_fn(module, inp, out):
            storage_dict[name] = out.detach().cpu()
        return hook_fn

    for i in range(10):                                
        handles = []
        for name in target_layers:
            module = dict(model.model.model.named_modules())[name]
            handles.append(module.register_forward_hook(
                get_hook(name, features_orig_dict[i])))

        _ = model.predict(img[i][None])                    

        for h in handles:                                  
            h.remove()

        handles = []
        for name in target_layers:
            module = dict(model.model.model.named_modules())[name]
            handles.append(module.register_forward_hook(
                get_hook(name, features_bright_dict[i])))

        _ = model.predict(img_bright[i][None])             

        for h in handles:
            h.remove()

In [ ]:
for path in patient_ids:
    for i in range(10):
        print(f"\n🧠 Slice {i}")
        for layer_name, tensor in features_orig_dict[i].items():
            channels = tensor.shape[1]
            print(f" {path} {layer_name}: {channels} channels")


In [ ]:
from skimage.metrics import structural_similarity as ssim
import numpy as np
for path in patient_ids:
    # Loop over all slices and all layers
    for i in range(10):
        print(f"\n######## Slice {i} ########")

        for layer_name in target_layers:
            fmap_orig   = features_orig_dict[i][layer_name]     # (1, C, H, W)
            fmap_bright = features_bright_dict[i][layer_name]   # (1, C, H, W)

            num_channels = fmap_orig.shape[1]
            print(f"\n=== Layer: {layer_name} | Channels: {num_channels} ===")

            for ch in range(num_channels):
                orig   = fmap_orig[0, ch].numpy()
                bright = fmap_bright[0, ch].numpy()

                orig_n   = (orig - orig.min()) / (orig.max() - orig.min() + 1e-8)
                bright_n = (bright - bright.min()) / (bright.max() - bright.min() + 1e-8)

                mad  = np.mean(np.abs(orig - bright))
                mse  = np.mean((orig - bright) ** 2)
                ssim_val = ssim(orig_n, bright_n, data_range=1)

                print(f"{path} | Slice {i} | {layer_name} | Ch {ch:02d} → MAD={mad:.4f}, MSE={mse:.4f}, SSIM={ssim_val:.4f}")


In [ ]:
with open("brightness_analysis_results_everything2.txt", "w") as f:
    for path in patient_ids:
        for i in range(10):
            f.write(f"\n######## Slice {i} ########\n")
            print(f"\n######## Slice {i} ########")

            for layer_name in target_layers:
                fmap_orig   = features_orig_dict[i][layer_name]     # (1, C, H, W)
                fmap_bright = features_bright_dict[i][layer_name]   # (1, C, H, W)

                num_channels = fmap_orig.shape[1]
                f.write(f"  \n=== path:{path} \n=== Layer: {layer_name} | Channels: {num_channels} ===\n")
                print(f"\n=== path:{path} \n=== Layer: {layer_name} | Channels: {num_channels} ===")

                for ch in range(num_channels):
                    orig   = fmap_orig[0, ch].numpy()
                    bright = fmap_bright[0, ch].numpy()

                    orig_n   = (orig - orig.min()) / (orig.max() - orig.min() + 1e-8)
                    bright_n = (bright - bright.min()) / (bright.max() - bright.min() + 1e-8)

                    mad  = np.mean(np.abs(orig - bright))
                    mse  = np.mean((orig - bright) ** 2)
                    ssim_val = ssim(orig_n, bright_n, data_range=1)

                    line = f"Slice {i} | {layer_name} | Ch {ch:02d} → MAD={mad:.4f}, MSE={mse:.4f}, SSIM={ssim_val:.4f}"
                    f.write(line + "\n")
                    #print(line)


In [ ]:
import csv
from skimage.metrics import structural_similarity as ssim

# Define your output file
with open("brightness_analysis_results.csv", mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["PatientID", "Slice", "Layer", "Channel", "MAD", "MSE", "SSIM"])

    for path in patient_ids:
        for i in range(10):
            for layer_name in target_layers:
                fmap_orig   = features_orig_dict[i][layer_name]     # (1, C, H, W)
                fmap_bright = features_bright_dict[i][layer_name]   # (1, C, H, W)

                num_channels = fmap_orig.shape[1]

                for ch in range(num_channels):
                    orig   = fmap_orig[0, ch].numpy()
                    bright = fmap_bright[0, ch].numpy()

                    # Normalize for SSIM
                    orig_n   = (orig - orig.min()) / (orig.max() - orig.min() + 1e-8)
                    bright_n = (bright - bright.min()) / (bright.max() - bright.min() + 1e-8)

                    mad  = np.mean(np.abs(orig - bright))
                    mse  = np.mean((orig - bright) ** 2)
                    ssim_val = ssim(orig_n, bright_n, data_range=1)

                    # Write row
                    writer.writerow([path, i, layer_name, ch, mad, mse, ssim_val])


In [ ]:
import re
import pandas as pd
with open("brightness_analysis_results_everything.txt", "r") as f:
            lines = f.readlines()
for path in patient_ids:
        pattern = re.compile(
            r"Slice (\d+) \| (.+?) \| Ch (\d+) → MAD=([0-9.]+), MSE=([0-9.]+), SSIM=([0-9.]+)"
        )

        data = []
        for line in lines:
            m = pattern.search(line)
            if m:
                data.append((
                    int(m.group(1)),     # slice index
                    m.group(2),          # layer name
                    int(m.group(3)),     # channel index
                    float(m.group(4)),   # MAD
                    float(m.group(5)),   # MSE
                    float(m.group(6))    # SSIM
                ))

        df = pd.DataFrame(data, columns=["Slice", "Layer", "Channel", "MAD", "MSE", "SSIM"])

        # === 2. Select top 3 channels with highest MAD per slice ===
        top3 = df.sort_values(["Slice", "MAD"], ascending=[True, False]).groupby("Slice").head(3)

        # === 3. Save feature map comparisons ===

        for _, row in top3.iterrows():
            slice_idx = row["Slice"]
            layer_name = row["Layer"]
            ch = row["Channel"]
            mad = row["MAD"]
            mse = row["MSE"]
            ssim_val = row["SSIM"]

            # Get feature maps
            orig = features_orig_dict[slice_idx][layer_name][0, ch].numpy()
            bright = features_bright_dict[slice_idx][layer_name][0, ch].numpy()
            abs_diff = np.abs(orig - bright)

            # Plot
            fig, axs = plt.subplots(1, 3, figsize=(12, 4))
            fig.suptitle(
                f"{path} - {layer_name} – Ch {ch}, Slice {slice_idx}\nMAD={mad:.4f} | MSE={mse:.4f} | SSIM={ssim_val:.4f}",
                fontsize=10
            )

            axs[0].imshow(orig, cmap="viridis")
            axs[0].set_title("Original")
            axs[0].axis("off")

            axs[1].imshow(bright, cmap="viridis")
            axs[1].set_title("Brightness Reduced")
            axs[1].axis("off")

            axs[2].imshow(abs_diff, cmap="magma")
            axs[2].set_title("Abs Diff")
            axs[2].axis("off")
            os.makedirs("slices_and_channels11-sinem", exist_ok=True)
            out_path = f"slices_and_channels11-sinem/{path}-layer_{layer_name.replace('.', '_')}_ch{ch}_slice{slice_idx}.png"
            plt.tight_layout()
            plt.savefig(out_path, dpi=150)
            plt.close()


In [ ]:
import re
for path in patient_ids:
    
    top3_by_layer = df.sort_values(["Layer", "MAD"], ascending=[True, False]).groupby("Layer").head(3)

    #os.makedirs("top3_per_layer", exist_ok=True)

    for _, row in top3_by_layer.iterrows():
        slice_idx = row["Slice"]
        layer_name = row["Layer"]
        ch = row["Channel"]
        mad = row["MAD"]
        mse = row["MSE"]
        ssim_val = row["SSIM"]

        # Get feature maps
        orig = features_orig_dict[slice_idx][layer_name][0, ch].numpy()
        bright = features_bright_dict[slice_idx][layer_name][0, ch].numpy()
        abs_diff = np.abs(orig - bright)

        # Plot
        fig, axs = plt.subplots(1, 3, figsize=(12, 4))
        fig.suptitle(
            f"{layer_name} – Ch {ch}, Slice {slice_idx}\nMAD={mad:.4f} | MSE={mse:.4f} | SSIM={ssim_val:.4f}",
            fontsize=10
        )

        axs[0].imshow(orig, cmap="viridis", interpolation="bilinear")
        axs[0].set_title("Original")
        axs[0].axis("off")

        axs[1].imshow(bright, cmap="viridis", interpolation="bilinear")
        axs[1].set_title("Brightness Reduced")
        axs[1].axis("off")

        axs[2].imshow(abs_diff, cmap="magma", interpolation="bilinear")
        axs[2].set_title("Abs Diff")
        axs[2].axis("off")
        os.makedirs("top3_per_layer_everything-sinem-new-09", exist_ok=True)
        out_path = f"top3_per_layer_everything-sinem-new-09/path_{path}_layer_{layer_name.replace('.', '_')}_ch{ch}_slice{slice_idx}.png"
        plt.tight_layout()
        plt.savefig(out_path, dpi=150)
        plt.close()


In [ ]:
import os
import re
import matplotlib.pyplot as plt
import numpy as np

# Bu listeyi dolduracağız
records = []

# TXT dosyasını oku ve her satırı işle
with open("brightness_analysis_results_everything.txt", "r") as f:
    for line in f:
        line = line.strip()
        if "→ MAD=" in line:
            match = re.match(
                r"(.*?)\s+\|\s+Slice\s+(\d+)\s+\|\s+(.*?)\s+\|\s+Ch\s+(\d+)\s+→ MAD=([\d.]+), MSE=([\d.]+), SSIM=([\d.]+)",
                line
            )
            if match:
                path, slice_idx, layer, ch, mad, mse, ssim_val = match.groups()
                records.append({
                    "Patient": path,
                    "Slice": int(slice_idx),
                    "Layer": layer,
                    "Channel": int(ch),
                    "MAD": float(mad),
                    "MSE": float(mse),
                    "SSIM": float(ssim_val)
                })

# En yüksek 3 MAD değerine göre katman başına filtreleme
import pandas as pd
df = pd.DataFrame(records)
top3_by_layer = df.sort_values(["Layer", "MAD"], ascending=[True, False]).groupby("Layer").head(3)

# Görselleri kaydet
for _, row in top3_by_layer.iterrows():
    path = row["Patient"]
    slice_idx = row["Slice"]
    layer_name = row["Layer"]
    ch = row["Channel"]
    mad = row["MAD"]
    mse = row["MSE"]
    ssim_val = row["SSIM"]

    orig = features_orig_dict[slice_idx][layer_name][0, ch].numpy()
    bright = features_bright_dict[slice_idx][layer_name][0, ch].numpy()
    abs_diff = np.abs(orig - bright)

    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    fig.suptitle(
        f"{layer_name} – Ch {ch}, Slice {slice_idx}, Patient {path}\nMAD={mad:.4f} | MSE={mse:.4f} | SSIM={ssim_val:.4f}",
        fontsize=10
    )

    axs[0].imshow(orig, cmap="viridis", interpolation="bilinear")
    axs[0].set_title("Original")
    axs[0].axis("off")

    axs[1].imshow(bright, cmap="viridis", interpolation="bilinear")
    axs[1].set_title("Brightness Reduced")
    axs[1].axis("off")

    axs[2].imshow(abs_diff, cmap="magma", interpolation="bilinear")
    axs[2].set_title("Abs Diff")
    axs[2].axis("off")

    os.makedirs("top3_per_layer_final", exist_ok=True)
    filename = f"top3_per_layer_final/{path}_layer_{layer_name.replace('.', '_')}_ch{ch}_slice{slice_idx}.png"
    plt.tight_layout()
    plt.savefig(filename, dpi=150)
    plt.close()


In [ ]:
import math

for path in patient_ids:
    img = all_images[path]           # (slices, H, W)
    gt = all_labels[path]            # (slices, H, W)

    # Brightness-adjusted input
    bright_img = img.copy()
    indices = np.where(gt == 3)
    bright_img[indices] *= 0.1

    # Predict on bright image
    pred_bright = model.predict(bright_img)         # (slices, classes, H, W)
    seg_bright = np.argmax(pred_bright, axis=1)     # (slices, H, W)

    num_slices = bright_img.shape[0]
    cols = 6
    rows = math.ceil(num_slices / cols)

    fig, axs = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))
    axs = axs.flatten()

    for i in range(num_slices):
        axs[i].imshow(bright_img[i], cmap="gray")
        axs[i].imshow(seg_bright[i], alpha=0.4)
        axs[i].set_title(f"Slice {i}", fontsize=8)
        axs[i].axis("off")

    # Turn off any unused subplots
    for i in range(num_slices, len(axs)):
        axs[i].axis("off")

    fig.suptitle(f"{path} – Brightness Adjusted Segmentation (All Slices)", fontsize=12)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()




In [ ]:
import os, re, nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

# --------------------------------------------------
# 0. CONFIG
# --------------------------------------------------
patient_ids = [
    "patient001_frame01", "patient001_frame12",
    "patient002_frame01", "patient002_frame12",
    "patient003_frame01", "patient003_frame15",
    "patient004_frame01", "patient004_frame15",
    "patient005_frame01", "patient005_frame13",
]

images_dir = "Task500_ACDC/imagesTr/"
labels_dir = "Task500_ACDC/labelsTr/"

master_rows = []   # for the global CSV

# --------------------------------------------------
# 1. MAIN LOOP
# --------------------------------------------------
for pid in patient_ids:
    print(f"\n=== Processing {pid} ===")

    # 1a. LOAD & TRANSPOSE
    img = nib.load(f"{images_dir}{pid}_0000.nii.gz").get_fdata().transpose(2,0,1)
    gt  = nib.load(f"{labels_dir}{pid}.nii.gz").get_fdata().transpose(2,0,1)

    # 1b. STYLE-ADJUST (brightness reduction on LV label 3)
    img_bright = img.copy()
    img_bright[gt == 3] *= 0.1

    # 1c. HOOK-BASED FEATURE EXTRACTION (first 10 slices)
    max_slices = min(10, img.shape[0])
    features_orig = [dict() for _ in range(max_slices)]
    features_bright = [dict() for _ in range(max_slices)]

    for i in range(max_slices):
        # ---- ORIGINAL ----
        handles = []
        for name in target_layers:
            mod = dict(model.model.model.named_modules())[name]
            handles.append(mod.register_forward_hook(get_hook(name, features_orig[i])))
        _ = model.predict(img[i][None])      # (1, H, W)
        for h in handles: h.remove()

        # ---- BRIGHTNESS-ADJ ----
        handles = []
        for name in target_layers:
            mod = dict(model.model.model.named_modules())[name]
            handles.append(mod.register_forward_hook(get_hook(name, features_bright[i])))
        _ = model.predict(img_bright[i][None])
        for h in handles: h.remove()

    # --------------------------------------------------
    # 2. METRIC CALC & LOG
    # --------------------------------------------------
    out_dir = f"results_{pid}"
    os.makedirs(out_dir,        exist_ok=True)
    os.makedirs(f"{out_dir}/top3_per_slice", exist_ok=True)
    os.makedirs(f"{out_dir}/top3_per_layer", exist_ok=True)

    # Collect metrics into DataFrame
    rows = []
    with open(f"{out_dir}/brightness_analysis_results.txt", "w") as f:
        for s in range(max_slices):
            f.write(f"\n######## Slice {s} ########\n")
            for layer in target_layers:
                fo = features_orig[s][layer]
                fb = features_bright[s][layer]
                C = fo.shape[1]
                f.write(f"\n=== Layer: {layer} | Channels: {C} ===\n")
                for ch in range(C):
                    o = fo[0, ch].numpy()
                    b = fb[0, ch].numpy()
                    on = (o - o.min())/(o.max()-o.min()+1e-8)
                    bn = (b - b.min())/(b.max()-b.min()+1e-8)
                    mad  = np.abs(o-b).mean()
                    mse  = ((o-b)**2).mean()
                    ssim_val = ssim(on, bn, data_range=1)
                    line = f"Slice {s} | {layer} | Ch {ch:02d} → MAD={mad:.4f}, MSE={mse:.4f}, SSIM={ssim_val:.4f}"
                    f.write(line+"\n")
                    rows.append([pid, s, layer, ch, mad, mse, ssim_val])

    df = pd.DataFrame(rows, columns=["Patient","Slice","Layer","Channel","MAD","MSE","SSIM"])

    # 2a. TOP-3 PER SLICE PNGs
    top3_slice = df.sort_values(["Slice","MAD"], ascending=[True,False]).groupby("Slice").head(3)
    for _, r in top3_slice.iterrows():
        s,l,ch,mad,mse,ssim_val = r["Slice"], r["Layer"], r["Channel"], r["MAD"], r["MSE"], r["SSIM"]
        o  = features_orig[s][l][0, ch].numpy()
        b  = features_bright[s][l][0, ch].numpy()
        ad = np.abs(o-b)
        fig,axs = plt.subplots(1,3,figsize=(12,4))
        fig.suptitle(f"{pid} • {l} • Ch{ch} • Slice{s}\nMAD={mad:.4f}",fontsize=9)
        axs[0].imshow(o, cmap="viridis", interpolation="bilinear");   axs[0].axis("off")
        axs[1].imshow(b, cmap="viridis", interpolation="bilinear");   axs[1].axis("off")
        axs[2].imshow(ad,cmap="magma",  interpolation="bilinear");    axs[2].axis("off")
        plt.tight_layout()
        plt.savefig(f"{out_dir}/top3_per_slice/{l.replace('.','_')}_ch{ch}_slice{s}.png", dpi=150)
        plt.close()

    # 2b. TOP-3 PER LAYER PNGs
    top3_layer = df.sort_values(["Layer","MAD"], ascending=[True,False]).groupby("Layer").head(3)
    for _, r in top3_layer.iterrows():
        s,l,ch,mad,mse,ssim_val = r["Slice"], r["Layer"], r["Channel"], r["MAD"], r["MSE"], r["SSIM"]
        o  = features_orig[s][l][0, ch].numpy()
        b  = features_bright[s][l][0, ch].numpy()
        ad = np.abs(o-b)
        fig,axs = plt.subplots(1,3,figsize=(12,4))
        fig.suptitle(f"{pid} • {l} • Ch{ch} • Slice{s}\nMAD={mad:.4f}",fontsize=9)
        axs[0].imshow(o, cmap="viridis", interpolation="bilinear");   axs[0].axis("off")
        axs[1].imshow(b, cmap="viridis", interpolation="bilinear");   axs[1].axis("off")
        axs[2].imshow(ad,cmap="magma",  interpolation="bilinear");    axs[2].axis("off")
        plt.tight_layout()
        plt.savefig(f"{out_dir}/top3_per_layer/{l.replace('.','_')}_ch{ch}_slice{s}.png", dpi=150)
        plt.close()

    # Append to master list
    master_rows.extend(rows)

# --------------------------------------------------
# 3. SAVE GLOBAL CSV
# --------------------------------------------------
master_df = pd.DataFrame(master_rows, columns=["Patient","Slice","Layer","Channel","MAD","MSE","SSIM"])
master_df.to_csv("all_mad_metrics.csv", index=False)
print("\n✓ Finished. See per-patient folders and all_mad_metrics.csv for details.")


In [ ]:

target_layers = [
    "0.conv.unit0.adn.N",
    "0.conv.unit1.adn.N",
    "0.conv.unit2.adn.N",
    "0.conv.unit3.adn.N",
    "0.conv.unit4.adn.N",
    "0.conv.unit5.adn.N",
    "0.conv.unit6.adn.N",
    "0.conv.unit7.adn.N",
    "1.submodule.0.conv.unit0.adn.N",
    "1.submodule.0.conv.unit1.adn.N",
    "1.submodule.0.conv.unit2.adn.N",
    "1.submodule.0.conv.unit3.adn.N",
    "1.submodule.0.conv.unit4.adn.N",
    "1.submodule.0.conv.unit5.adn.N",
    "1.submodule.0.conv.unit6.adn.N",
    "1.submodule.0.conv.unit7.adn.N",
    "1.submodule.1.submodule.0.conv.unit0.adn.N",
    "1.submodule.1.submodule.0.conv.unit1.adn.N",
    "1.submodule.1.submodule.0.conv.unit2.adn.N",
    "1.submodule.1.submodule.0.conv.unit3.adn.N",
    "1.submodule.1.submodule.0.conv.unit4.adn.N",
    "1.submodule.1.submodule.0.conv.unit5.adn.N",
    "1.submodule.1.submodule.0.conv.unit6.adn.N",
    "1.submodule.1.submodule.0.conv.unit7.adn.N",
    "1.submodule.1.submodule.1.submodule.0.conv.unit0.adn.N",
    "1.submodule.1.submodule.1.submodule.0.conv.unit1.adn.N",
    "1.submodule.1.submodule.1.submodule.0.conv.unit2.adn.N",
    "1.submodule.1.submodule.1.submodule.0.conv.unit3.adn.N",
    "1.submodule.1.submodule.1.submodule.0.conv.unit4.adn.N",
    "1.submodule.1.submodule.1.submodule.0.conv.unit5.adn.N",
    "1.submodule.1.submodule.1.submodule.0.conv.unit6.adn.N",
    "1.submodule.1.submodule.1.submodule.0.conv.unit7.adn.N",
    "1.submodule.1.submodule.2.0.adn.N",
    "1.submodule.1.submodule.2.1.conv.unit0.adn.N",
    "1.submodule.2.0.adn.N",
    "1.submodule.2.1.conv.unit0.adn.N"
]

features_orig_dict = {}
features_bright_dict = {}

img_bright = style_adjusted_input.copy()

def get_hook(name, storage_dict):
    def hook_fn(module, input, output):
        storage_dict[name] = output.detach().cpu()
    return hook_fn

handles = []
for name in target_layers:
    module = dict(model.model.model.named_modules())[name]
    h = module.register_forward_hook(get_hook(name, features_orig_dict))
    handles.append(h)
_ = model.predict(img)
for h in handles:
    h.remove()

handles = []
for name in target_layers:
    module = dict(model.model.model.named_modules())[name]
    h = module.register_forward_hook(get_hook(name, features_bright_dict))
    handles.append(h)

_ = model.predict(img_bright)
for h in handles:
    h.remove()

In [ ]:
channels_to_check = range(16)
slice_idx = 5  

for layer_name in target_layers:
    fmap_orig = features_orig_dict[layer_name]
    fmap_bright = features_bright_dict[layer_name]

    print(f"\n===== Layer: {layer_name} =====")

    for ch in channels_to_check:
        orig = fmap_orig[0, ch, :, :].numpy()  
        bright = fmap_bright[0, ch, :, :].numpy()

        orig_norm = (orig - orig.min()) / (orig.max() - orig.min() + 1e-8)
        bright_norm = (bright - bright.min()) / (bright.max() - bright.min() + 1e-8)

        abs_diff = np.abs(orig - bright)
        mad = np.mean(abs_diff)
        mse = np.mean((orig - bright) ** 2)
        ssim_val = ssim(orig_norm, bright_norm, data_range=1)

        print(f"Channel {ch}: MAD={mad:.4f}, MSE={mse:.4f}, SSIM={ssim_val:.4f}")

        fig, axs = plt.subplots(1, 3, figsize=(12, 4))
        fig.suptitle(f"{layer_name} – Channel {ch}\nMAD={mad:.4f} | MSE={mse:.4f} | SSIM={ssim_val:.4f}", fontsize=10)

        axs[0].imshow(orig, cmap="viridis")
        axs[0].set_title("Original")
        axs[0].axis("off")

        axs[1].imshow(bright, cmap="viridis")
        axs[1].set_title("Brightness Reduced")
        axs[1].axis("off")

        axs[2].imshow(abs_diff, cmap="magma")
        axs[2].set_title("Abs Diff")
        axs[2].axis("off")

        os.makedirs("slices_and_channels", exist_ok=True)
        plt.tight_layout()
        out_path = f"slices_and_channels/layer_{layer_name.replace('.', '_')}_ch{ch}_slice{slice_idx}.png"
        plt.savefig(out_path)
        plt.close()

In [ ]:
num_slices = fmap_orig.shape[2]
print(f"{layer_name} → Number of slices: {num_slices}")


In [ ]:
channels_to_check = range(16)  # Adjust if needed

for layer_name in target_layers:
    fmap_orig = features_orig_dict[layer_name]
    fmap_bright = features_bright_dict[layer_name]

    num_slices = fmap_orig.shape[2]  # depth axis
    slices_to_check = range(num_slices)

    print(f"\n===== Layer: {layer_name} → Number of slices: {num_slices} =====")

    for ch in channels_to_check:
        for slice_idx in slices_to_check:
            orig = fmap_orig[0, ch, :, :].numpy()
            bright = fmap_bright[0, ch, :, :].numpy()

            orig_norm = (orig - orig.min()) / (orig.max() - orig.min() + 1e-8)
            bright_norm = (bright - bright.min()) / (bright.max() - bright.min() + 1e-8)

            abs_diff = np.abs(orig - bright)
            mad = np.mean(abs_diff)
            mse = np.mean((orig - bright) ** 2)
            ssim_val = ssim(orig_norm, bright_norm, data_range=1)

            print(f"Layer {layer_name} | Channel {ch} | Slice {slice_idx} → MAD={mad:.4f}, MSE={mse:.4f}, SSIM={ssim_val:.4f}")

            fig, axs = plt.subplots(1, 3, figsize=(12, 4))
            fig.suptitle(
                f"{layer_name} – Ch {ch}, Slice {slice_idx}\nMAD={mad:.4f} | MSE={mse:.4f} | SSIM={ssim_val:.4f}",
                fontsize=10
            )

            axs[0].imshow(orig, cmap="viridis")
            axs[0].set_title("Original")
            axs[0].axis("off")

            axs[1].imshow(bright, cmap="viridis")
            axs[1].set_title("Brightness Reduced")
            axs[1].axis("off")

            axs[2].imshow(abs_diff, cmap="magma")
            axs[2].set_title("Abs Diff")
            axs[2].axis("off")

            os.makedirs("slices_and_channels", exist_ok=True)
            out_path = f"slices_and_channels/layer_{layer_name.replace('.', '_')}_ch{ch}_slice{slice_idx}.png"
            plt.tight_layout()
            plt.savefig(out_path)
            plt.close()

In [ ]:
channels_to_check = range(16)  # Adjust if needed

for layer_name in target_layers:
    fmap_orig = features_orig_dict[layer_name]
    fmap_bright = features_bright_dict[layer_name]

    num_slices = fmap_orig.shape[2]  # depth axis
    slices_to_check = range(num_slices)

    print(f"\n===== Layer: {layer_name} → Number of slices: {num_slices} =====")

    for ch in channels_to_check:
        for slice_idx in slices_to_check:
            orig = fmap_orig[0, ch, :, :].numpy()
            bright = fmap_bright[0, ch, :, :].numpy()

            orig_norm = (orig - orig.min()) / (orig.max() - orig.min() + 1e-8)
            bright_norm = (bright - bright.min()) / (bright.max() - bright.min() + 1e-8)

            abs_diff = np.abs(orig - bright)
            mad = np.mean(abs_diff)
            mse = np.mean((orig - bright) ** 2)
            ssim_val = ssim(orig_norm, bright_norm, data_range=1)

            print(f"Layer {layer_name} | Channel {ch} | Slice {slice_idx} → MAD={mad:.4f}, MSE={mse:.4f}, SSIM={ssim_val:.4f}")

            fig, axs = plt.subplots(1, 3, figsize=(12, 4))
            fig.suptitle(
                f"{layer_name} – Ch {ch}, Slice {slice_idx}\nMAD={mad:.4f} | MSE={mse:.4f} | SSIM={ssim_val:.4f}",
                fontsize=10
            )

            axs[0].imshow(orig, cmap="viridis")
            axs[0].set_title("Original")
            axs[0].axis("off")

            axs[1].imshow(bright, cmap="viridis")
            axs[1].set_title("Brightness Reduced")
            axs[1].axis("off")

            axs[2].imshow(abs_diff, cmap="magma")
            axs[2].set_title("Abs Diff")
            axs[2].axis("off")

In [ ]:
for layer_name in target_layers:
    fmap_orig = features_orig_dict[layer_name]
for name, fmap in features_orig_dict.items():
    print(f"{name} → {fmap.shape}")

In [ ]:
# --- Görselleştirme (veya kayıt) ---
# Değişkenler zaten bellekte:
#   patient_ids, all_images, all_labels, all_segmentations

for path in patient_ids:
    img     = all_images[path]          # (slices, H, W)
    gt      = all_labels[path]          # (slices, H, W)
    result  = all_segmentations[path]   # (slices, H, W)

    for slide in range(img.shape[0]):
        plt.figure(figsize=(12, 4))

        # Orijinal
        plt.subplot(1, 3, 1)
        plt.imshow(img[slide], cmap="gray")
        plt.title("Original")
        plt.axis("off")

        # Ground truth
        plt.subplot(1, 3, 2)
        plt.imshow(img[slide], cmap="gray")
        plt.imshow(gt[slide], alpha=0.4)
        plt.title("Ground Truth")
        plt.axis("off")

        # Segmentasyon sonucu
        plt.subplot(1, 3, 3)
        plt.imshow(img[slide], cmap="gray")
        plt.imshow(result[slide], alpha=0.4)
        plt.title("Segmentation")
        plt.axis("off")

        plt.suptitle(f"{path} – Slice {slide}", fontsize=10)
        plt.tight_layout()
        plt.show()        # yerine kaydetmek istersen: plt.savefig(...); plt.close()


In [ ]:
import math

for path in patient_ids:
    img = all_images[path]           # (slices, H, W)
    gt = all_labels[path]            # (slices, H, W)

    # Brightness-adjusted input
    bright_img = img.copy()
    indices = np.where(gt == 3)
    bright_img[indices] *= 0.1

    # Predict on bright image
    pred_bright = model.predict(bright_img)         # (slices, classes, H, W)
    seg_bright = np.argmax(pred_bright, axis=1)     # (slices, H, W)

    num_slices = bright_img.shape[0]
    cols = 6
    rows = math.ceil(num_slices / cols)

    fig, axs = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))
    axs = axs.flatten()

    for i in range(num_slices):
        axs[i].imshow(bright_img[i], cmap="gray")
        axs[i].imshow(seg_bright[i], alpha=0.4)
        axs[i].set_title(f"Slice {i}", fontsize=8)
        axs[i].axis("off")

    # Turn off any unused subplots
    for i in range(num_slices, len(axs)):
        axs[i].axis("off")

    fig.suptitle(f"{path} – Brightness Adjusted Segmentation (All Slices)", fontsize=12)
    
    plt.show()


In [ ]:
from monai.metrics import DiceMetric
from torch.nn.functional import one_hot
import torch

def to_onehot(arr: np.ndarray, num_classes: int) -> torch.Tensor:
    return one_hot(torch.from_numpy(arr).long(), num_classes).permute(0, 3, 1, 2).float()

dice_metric = DiceMetric(include_background=True, reduction="mean_batch")
dice_scores_bright = {}

for path in patient_ids:
    print(f"\n▶ Segmenting and evaluating: {path}")

    img = all_images[path]        # (slices, H, W)
    gt = all_labels[path]         # (slices, H, W)

    # Brightness adjustment (LV = label 3)
    bright_img = img.copy()
    indices = np.where(gt == 3)
    bright_img[indices] *= 0.1

    # Segment
    pred_bright = model.predict(bright_img)           # (slices, classes, H, W)
    seg_bright = np.argmax(pred_bright, axis=1)       # (slices, H, W)

    # Dice calculation
    num_classes = pred_bright.shape[1]
    gt_onehot = to_onehot(gt, num_classes)
    pred_onehot = to_onehot(seg_bright, num_classes)

    score = dice_metric(pred_onehot, gt_onehot).item()
    dice_scores_bright[path] = score
    dice_metric.reset()

# === Print results ===
print("\n=== Dice Scores (Brightness Adjusted Input) ===")
for path, score in dice_scores_bright.items():
    print(f"{path}: {score:.4f}")

mean_dice = np.mean(list(dice_scores_bright.values()))
print(f"\n▶ Mean Dice Score: {mean_dice:.4f}")


In [ ]:
import numpy as np
import torch
from monai.metrics import DiceMetric
from torch.nn.functional import one_hot

# === Yardımcı fonksiyon ===
def to_onehot(arr: np.ndarray, num_classes: int) -> torch.Tensor:
    return one_hot(torch.from_numpy(arr).long(), num_classes).permute(0, 3, 1, 2).float()

# === Dice metriği ===
dice_metric = DiceMetric(include_background=True, reduction="none")

dice_vs_gt_orig = {}
dice_vs_gt_bright = {}
dice_internal_consistency = {}

for path in patient_ids:
    print(f"\n▶ Processing: {path}")
    
    img = all_images[path]          # (slices, H, W)
    gt  = all_labels[path]          # (slices, H, W)

    # === 1. Orijinal input ile segmentasyon ===
    pred_orig = model.predict(img)                     # (slices, classes, H, W)
    seg_orig = np.argmax(pred_orig, axis=1)            # (slices, H, W)

    # === 2. Brightness adjustment ===
    bright_img = img.copy()
    bright_img[np.where(gt == 3)] *= 0.1

    # === 3. Bright input ile segmentasyon ===
    pred_bright = model.predict(bright_img)
    seg_bright = np.argmax(pred_bright, axis=1)

    # === 4. One-hot formatına çevir ===
    num_classes = pred_orig.shape[1]
    gt_onehot = to_onehot(gt, num_classes)
    seg_orig_onehot = to_onehot(seg_orig, num_classes)
    seg_bright_onehot = to_onehot(seg_bright, num_classes)

    # === 5. Dice hesaplamaları ===
    score_gt_orig = dice_metric(seg_orig_onehot, gt_onehot).mean().item()
    score_gt_bright = dice_metric(seg_bright_onehot, gt_onehot).mean().item()
    score_consistency = dice_metric(seg_orig_onehot, seg_bright_onehot).mean().item()

    # === 6. Kaydet ===
    dice_vs_gt_orig[path] = score_gt_orig
    dice_vs_gt_bright[path] = score_gt_bright
    dice_internal_consistency[path] = score_consistency
    dice_metric.reset()

# === SONUÇLAR ===
print("\n=== Dice vs Ground Truth ===")
for path in patient_ids:
    print(f"{path}: Original = {dice_vs_gt_orig[path]:.4f} | Bright = {dice_vs_gt_bright[path]:.4f}")

print("\n=== Internal Consistency (Original vs Bright Prediction) ===")
for path in patient_ids:
    print(f"{path}: Dice = {dice_internal_consistency[path]:.4f}")

# Ortalamaları yaz
mean_gt_orig = np.mean(list(dice_vs_gt_orig.values()))
mean_gt_bright = np.mean(list(dice_vs_gt_bright.values()))
mean_consistency = np.mean(list(dice_internal_consistency.values()))

print(f"\n▶ Mean Dice vs GT – Original: {mean_gt_orig:.4f}")
print(f"▶ Mean Dice vs GT – Bright  : {mean_gt_bright:.4f}")
print(f"▶ Mean Internal Consistency : {mean_consistency:.4f}")


In [ ]:
def dice_score(pred_mask, true_mask, class_id=3):
    pred_bin = (pred_mask == class_id).astype(np.uint8)
    true_bin = (true_mask == class_id).astype(np.uint8)

    intersection = np.sum(pred_bin * true_bin)
    total = np.sum(pred_bin) + np.sum(true_bin)

    return 1.0 if total == 0 else 2.0 * intersection / total

print("Patient ID             Slice    Dice Orig    Dice Bright")
print("----------------------------------------------------------")

for path in patient_ids:
    img = all_images[path]                 # (slices, H, W)
    gt  = all_labels[path]                 # (slices, H, W)
    seg_orig = all_segmentations[path]     # (slices, H, W)

    # Predict on adjusted
    pred_bright = model.predict(style_adjusted_input)
    seg_bright = np.argmax(pred_bright, axis=1)

    for i in range(img.shape[0]):
        dice_o = dice_score(seg_orig[i], gt[i], class_id=3)
        pred_slice = seg_bright[i]
        gt_slice = gt[i]
        if pred_slice.shape != gt_slice.shape:
            pred_slice = pred_slice.T
        dice_b = dice_score(pred_slice, gt_slice, class_id=3)

        print(f"{path:<23} {i:<8} {dice_o:<12.4f} {dice_b:<12.4f}")


In [ ]:
for path in patient_ids:
    print(f"\n▶ {path}")
    img = all_images[path]
    gt  = all_labels[path]

    for i in range(img.shape[0]):
        if img[i].shape != gt[i].shape:
            print(f"❌ Mismatch at slice {i}: image = {img[i].shape}, gt = {gt[i].shape}")


In [ ]:
def dice_score(pred_mask, true_mask, class_id=3):
    pred_bin = (pred_mask == class_id).astype(np.uint8)
    true_bin = (true_mask == class_id).astype(np.uint8)

    intersection = np.sum(pred_bin * true_bin)
    total = np.sum(pred_bin) + np.sum(true_bin)

    return 1.0 if total == 0 else 2.0 * intersection / total

# --- Görselleştirme + Dice Skorları ---
for path in patient_ids:
    img     = all_images[path]          # (slices, H, W)
    gt      = all_labels[path]          # (slices, H, W)
    result  = all_segmentations[path]   # (slices, H, W)

    for slide in range(img.shape[0]):
        gt_slice = gt[slide]
        pred_slice = result[slide]

        # Boyut uyuşmazlığı varsa kırp
        if gt_slice.shape != pred_slice.shape:
            h = min(gt_slice.shape[0], pred_slice.shape[0])
            w = min(gt_slice.shape[1], pred_slice.shape[1])
            gt_slice = gt_slice[:h, :w]
            pred_slice = pred_slice[:h, :w]

        dice = dice_score(pred_slice, gt_slice, class_id=3)
        print(f"{path} – Slice {slide}: Dice Score = {dice:.4f}")

        # Görsel gösterim (opsiyonel)
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img[slide], cmap="gray")
        plt.title("Original")
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.imshow(img[slide], cmap="gray")
        plt.imshow(gt_slice, alpha=0.4)
        plt.title("Ground Truth")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.imshow(img[slide], cmap="gray")
        plt.imshow(pred_slice, alpha=0.4)
        plt.title("Segmentation")
        plt.axis("off")

        plt.suptitle(f"{path} – Slice {slide}", fontsize=10)
        plt.tight_layout()
        plt.show()


In [ ]:
 import math
import matplotlib.pyplot as plt
import numpy as np

for path in patient_ids:
    img = all_images[path]           # shape: (slices, H, W)
    gt = all_labels[path]           # shape: (slices, H, W)

    # Brightness-adjusted input
    bright_img = img.copy()
    indices = np.where(gt == 3)
    bright_img[indices] *= 0.1

    # Predict
    pred_bright = model.predict(bright_img)         # shape: (slices, classes, H, W)
    seg_bright = np.argmax(pred_bright, axis=1)     # shape: (slices, H, W)

    num_slices = bright_img.shape[0]
    cols = 3
    rows = num_slices

    fig, axs = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))

    if rows == 1:
        axs = np.expand_dims(axs, axis=0)  # handle single-slice case

    for i in range(num_slices):
        plt.suptitle(f"{path} – Slice {num_slices}", fontsize=10)
        axs[i, 0].imshow(img[i], cmap="gray")
        axs[i, 0].set_title("Original", fontsize=8)
        axs[i, 0].axis("off")

        axs[i, 1].imshow(img[i], cmap="gray")
        axs[i, 1].imshow(gt[i], alpha=0.4)
        axs[i, 1].set_title("Ground Truth", fontsize=8)
        axs[i, 1].axis("off")

        axs[i, 2].imshow(img[i], cmap="gray")
        axs[i, 2].imshow(seg_bright[i], alpha=0.4)
        axs[i, 2].set_title("Segmentation", fontsize=8)
        axs[i, 2].axis("off")
        

    #fig.suptitle(f"{path} – Brightness Adjusted Segmentation", fontsize=12)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


In [ ]:
all_predictions_bright = {}
all_segmentations_bright = {}

for path in patient_ids:
    print(f"\n▶ Performing segmentation for: {path}")
    # perform segmentation
    pred_bright = model.predict(style_adjusted_input)  # shape: (slices, classes, H, W)
    print(np.shape(pred_bright))

    result_bright = np.argmax(pred, axis=1)  # shape: (slices, H, W)
    print(np.shape(result_bright))

    all_predictions_bright[path] = pred_bright
    all_segmentations_bright[path] = result_bright


    for slide in range(style_adjusted_input.shape[0]):
        plt.figure(figsize=(12, 4))

        # Adjusted input
        plt.subplot(1, 3, 1)
        plt.imshow(style_adjusted_input[slide], cmap="gray")
        plt.title("Brightness-Adjusted Input")
        plt.axis("off")

        # Ground truth overlay
        plt.subplot(1, 3, 2)
        plt.imshow(style_adjusted_input[slide], cmap="gray")
        plt.imshow(gt[slide], alpha=0.4)
        plt.title("GT Overlay")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.imshow(style_adjusted_input[slide], cmap="gray")
        plt.imshow(result_bright[slide], alpha=0.4)
        plt.title("Prediction on Adjusted")
        plt.axis("off")

        plt.suptitle(f"{path} – Bright Slice {slide}", fontsize=10)
        plt.tight_layout()
        plt.show()

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np

for path in patient_ids:
    img = all_images[path]           # shape: (slices, H, W)
    gt = all_labels[path]            # shape: (slices, H, W)

    bright_img = img.copy()
    indices = np.where(gt == 3)
    bright_img[indices] *= 0.1

    pred_bright = model.predict(bright_img)         # shape: (slices, classes, H, W)
    seg_bright = np.argmax(pred_bright, axis=1)     # shape: (slices, H, W)

    num_slices = bright_img.shape[0]
    cols = 3
    rows = num_slices

    fig, axs = plt.subplots(rows, cols, figsize=(cols * 3.2, rows * 3.2))
    if rows == 1:
        axs = np.expand_dims(axs, axis=0)

    for i in range(num_slices):
        # Her satırın üstüne yazı ekle
        axs[i, 0].set_ylabel(f"{path}\nSlice {i}", fontsize=9, rotation=0, labelpad=70, va='center')

        axs[i, 0].imshow(img[i], cmap="gray")
        axs[i, 0].set_title("Original", fontsize=8)
        axs[i, 0].axis("off")

        axs[i, 1].imshow(img[i], cmap="gray")
        axs[i, 1].imshow(gt[i], alpha=0.4)
        axs[i, 1].set_title("Ground Truth", fontsize=8)
        axs[i, 1].axis("off")

        axs[i, 2].imshow(img[i], cmap="gray")
        axs[i, 2].imshow(seg_bright[i], alpha=0.4)
        axs[i, 2].set_title("Segmentation", fontsize=8)
        axs[i, 2].axis("off")

    plt.tight_layout()
    plt.show()


In [ ]:
def dice_score(pred_mask, true_mask, class_id=3):
    pred_bin = (pred_mask == class_id).astype(np.uint8)
    true_bin = (true_mask == class_id).astype(np.uint8)
    intersection = np.sum(pred_bin * true_bin)
    total = np.sum(pred_bin) + np.sum(true_bin)
    return 1.0 if total == 0 else 2.0 * intersection / total


In [ ]:
for path in patient_ids:
    print(f"\n▶ Dice scores for: {path}")
  # prediction on bright input
    if gt[slide].shape != result_bright[slide].shape:
        gt_slice = gt[slide].T  # Transpose ground truth
    else:
        gt_slice = gt[slide]
    
    for slide in range(style_adjusted_input.shape[0]):
        score = dice_score(result_bright[slide], gt[slide], class_id=3)
        print(f"Slice {slide:02d} – Dice: {score:.4f}")


In [ ]:
print(f"GT shape: {gt.shape}, Prediction shape: {result_bright.shape}")
print(f"One slice GT: {gt[0].shape}, One slice prediction: {result_bright[0].shape}")


In [ ]:
for path in patient_ids:
    print(f"\n▶ Dice scores for: {path}")
    gt = all_labels[path]                            # (slices, 216, 256)
    result_bright = all_segmentations_bright[path]   # (slices, 256, 216)

    for slide in range(result_bright.shape[0]):
        pred_slice = result_bright[slide]            # (256, 216)
        gt_slice = gt[slide].T                       # transpose GT to match prediction

        score = dice_score(pred_slice, gt_slice, class_id=3)
        print(f"Slice {slide:02d} – Dice: {score:.4f}")


In [ ]:
from skimage.transform import resize

for path in patient_ids:
    print(f"\n▶ Dice scores for: {path}")
    gt = all_labels[path]                            # (slices, H, W)
    result_bright = all_segmentations_bright[path]   # (slices, H2, W2)

    for slide in range(result_bright.shape[0]):
        pred_slice = result_bright[slide]
        gt_slice = gt[slide]

        if pred_slice.shape != gt_slice.shape:
            print(f"⚠️ Resizing prediction for {path}, slice {slide}")
            pred_slice = resize(
                pred_slice,
                gt_slice.shape,
                order=0,  # Nearest-neighbor to preserve label classes
                preserve_range=True,
                anti_aliasing=False
            ).astype(np.uint8)

        score = dice_score(pred_slice, gt_slice, class_id=3)
        print(f"Slice {slide:02d} – Dice: {score:.4f}")


In [ ]:
orig_means = [np.mean(dice_scores_original[p]) for p in patient_ids]
bright_means = [np.mean(dice_scores_bright[p]) for p in patient_ids]

plt.figure(figsize=(6, 6))
plt.scatter(orig_means, bright_means, color="blue")
plt.plot([0, 1], [0, 1], 'k--', label="x = y")
plt.xlabel("Original Dice Mean")
plt.ylabel("Bright-Adjusted Dice Mean")
plt.title("Patient-wise Mean Dice Comparison")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
from skimage.transform import resize

dice_scores_original = {}
dice_scores_bright = {}

for path in patient_ids:
    gt = all_labels[path]
    result_orig = all_segmentations[path]
    result_bright = all_segmentations_bright[path]

    orig_scores = []
    bright_scores = []

    for slide in range(gt.shape[0]):
        gt_slice = gt[slide]
        pred_orig = result_orig[slide]
        pred_bright = result_bright[slide]

        # --- Resize if needed (for both)
        if gt_slice.shape != pred_orig.shape:
            h = min(gt_slice.shape[0], pred_orig.shape[0])
            w = min(gt_slice.shape[1], pred_orig.shape[1])
            gt_slice = gt_slice[:h, :w]
            pred_orig = pred_orig[:h, :w]

        if pred_bright.shape != gt_slice.shape:
            pred_bright = resize(pred_bright, gt_slice.shape, order=0, preserve_range=True, anti_aliasing=False).astype(np.uint8)

        # --- Dice scores for class 3
        d1 = dice_score(pred_orig, gt_slice, class_id=3)
        d2 = dice_score(pred_bright, gt_slice, class_id=3)

        orig_scores.append(d1)
        bright_scores.append(d2)

    dice_scores_original[path] = orig_scores
    dice_scores_bright[path] = bright_scores


In [ ]:
plt.figure(figsize=(6, 5))
plt.boxplot([all_orig, all_bright], labels=["Original", "Bright-Adjusted"], showmeans=True)
plt.ylabel("Dice Score")
plt.title("Boxplot – Dice Scores per Condition (Class 3)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
for path in patient_ids:
    orig = dice_scores_original[path]
    bright = dice_scores_bright[path]

    plt.figure(figsize=(8, 4))
    plt.plot(orig, label="Original", marker="o")
    plt.plot(bright, label="Bright-Adjusted", marker="x")
    plt.title(f"{path} – Dice Score per Slice")
    plt.xlabel("Slice Index")
    plt.ylabel("Dice Score")
    plt.ylim(0, 1)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()
    output_dir = "dice_core_plots"
    os.makedirs(output_dir, exist_ok=True)
    filename = os.path.join(output_dir, f"{path}_dice_comparison.png")
    plt.savefig(filename)
    plt.close()


In [ ]:
for path in patient_ids:
    print(path)
    print(dice_scores_bright[path])  # or dice_scores_original[path]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

all_deltas = []

for path in patient_ids:
    orig = np.array(dice_scores_original[path])
    bright = np.array(dice_scores_bright[path])
    delta = bright - orig
    all_deltas.extend(delta)

plt.figure(figsize=(8, 4))
plt.hist(all_deltas, bins=20, color="purple", alpha=0.7)
plt.axvline(0, color="black", linestyle="--")
plt.title("Histogram of Dice Δ (Bright - Original)")
plt.xlabel("Δ Dice Score")
plt.ylabel("Frequency")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
def plot_cdf(data, label, color):
    sorted_data = np.sort(data)
    cdf = np.arange(len(data)) / float(len(data))
    plt.plot(sorted_data, cdf, label=label, color=color)

all_orig = np.concatenate([dice_scores_original[p] for p in patient_ids])
all_bright = np.concatenate([dice_scores_bright[p] for p in patient_ids])

plt.figure(figsize=(8, 5))
plot_cdf(all_orig, "Original", "blue")
plot_cdf(all_bright, "Bright-Adjusted", "orange")
plt.title("CDF of Dice Scores (Class 3)")
plt.xlabel("Dice Score")
plt.ylabel("Cumulative Probability")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
if np.array_equal(pred_orig, pred_bright):
    print(f"⚠️ WARNING: {path} slice {slide} – bright and original predictions are IDENTICAL")


In [ ]:
delta_means = [np.mean(dice_scores_bright[p]) - np.mean(dice_scores_original[p]) for p in patient_ids]

plt.figure(figsize=(8, 4))
plt.bar(patient_ids, delta_means, color="green")
plt.axhline(0, linestyle="--", color="black")
plt.title("Mean Dice Δ per Patient (Bright - Original)")
plt.ylabel("Δ Dice Score")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
means_orig = [np.mean(dice_scores_original[p]) for p in patient_ids]
means_bright = [np.mean(dice_scores_bright[p]) for p in patient_ids]

plt.figure(figsize=(6, 5))
plt.boxplot([means_orig, means_bright], labels=["Original", "Bright-Adjusted"], showmeans=True)
plt.title("Per-Patient Mean Dice (Class 3)")
plt.ylabel("Mean Dice Score")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import os

output_dir = "dice_score_plots"
os.makedirs(output_dir, exist_ok=True)

for path in patient_ids:
    orig = dice_scores_original[path]
    bright = dice_scores_bright[path]

    plt.figure(figsize=(8, 4))
    plt.plot(orig, label="Original", marker="o")
    plt.plot(bright, label="Bright-Adjusted", marker="x")
    plt.title(f"{path} – Dice Score per Slice")
    plt.xlabel("Slice Index")
    plt.ylabel("Dice Score")
    plt.ylim(0, 1)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    
    filename = os.path.join(output_dir, f"{path}_dice_comparison.png")
    plt.savefig(filename)
    plt.close()


In [ ]:
import re
import pandas as pd

txt_path = "brightness_analysis_results_everything2.txt"

data = []

with open(txt_path, "r") as f:
    current_path = None
    current_slice = None

    for line in f:
        line = line.strip()

        # Match path line
        if line.startswith("=== path:"):
            current_path = line.split("path:")[1].strip()

        # Match slice line
        elif line.startswith("######## Slice"):
            match = re.search(r"Slice (\d+)", line)
            if match:
                current_slice = int(match.group(1))

        # Match channel entries
        elif "→ MAD=" in line:
            match = re.match(r"Slice (\d+) \| (.+?) \| Ch (\d+)", line)
            if match:
                slice_in_line = int(match.group(1))
                layer = match.group(2).strip()
                channel = int(match.group(3))

                mad_match = re.search(r"MAD=([\d.]+)", line)
                mse_match = re.search(r"MSE=([\d.]+)", line)
                ssim_match = re.search(r"SSIM=([\d.]+)", line)

                mad = float(mad_match.group(1)) if mad_match else None
                mse = float(mse_match.group(1)) if mse_match else None
                ssim = float(ssim_match.group(1)) if ssim_match else None

                data.append({
                    "Patient": current_path,
                    "Slice": slice_in_line,
                    "Layer": layer,
                    "Channel": channel,
                    "MAD": mad,
                    "MSE": mse,
                    "SSIM": ssim
                })

# Create dataframe
df = pd.DataFrame(data)

# Check the shape and a sample
print(f"✅ Loaded {len(df)} rows.")
display(df.head(20))


In [ ]:
top3_by_layer = df.sort_values(["Layer", "MAD"], ascending=[True, False]).groupby("Layer").head(3)


In [ ]:
import re
from collections import defaultdict

# Load the raw text
with open("brightness_analysis_results_everything2.txt", "r") as file:
    lines = file.readlines()

# Storage for parsed entries
entries = []

current_slice = None
current_layer = None
current_path = None

# Regular expressions
path_re = re.compile(r"=== path:(.+)")
layer_re = re.compile(r"=== Layer: (.+) \| Channels: (\d+)")
line_re = re.compile(r"Slice (\d+) \| (.+) \| Ch (\d+) → MAD=(\d+\.\d+), MSE=(\d+\.\d+), SSIM=(\d+\.\d+)")

# Parse the file
for line in lines:
    path_match = path_re.match(line)
    if path_match:
        current_path = path_match.group(1).strip()
        continue

    layer_match = layer_re.match(line)
    if layer_match:
        current_layer = layer_match.group(1).strip()
        continue

    data_match = line_re.match(line)
    if data_match:
        slice_idx = int(data_match.group(1))
        layer_name = data_match.group(2).strip()
        ch = int(data_match.group(3))
        mad = float(data_match.group(4))
        mse = float(data_match.group(5))
        ssim = float(data_match.group(6))

        entries.append({
            "Patient": current_path,
            "Slice": slice_idx,
            "Layer": layer_name,
            "Channel": ch,
            "MAD": mad,
            "MSE": mse,
            "SSIM": ssim
        })

# Group by layer
layer_to_entries = defaultdict(list)
for e in entries:
    layer_to_entries[e["Layer"]].append(e)

# Print top 3 MAD entries per layer, including ties
print("Top 3 MAD entries per layer (including ties):\n")

for layer, items in layer_to_entries.items():
    sorted_items = sorted(items, key=lambda x: -x["MAD"])

    top_mads = sorted(set([x["MAD"] for x in sorted_items]), reverse=True)[:3]
    top_entries = [e for e in sorted_items if e["MAD"] in top_mads]

    print(f"\n=== Layer: {layer} | Top MADs: {', '.join(f'{m:.4f}' for m in top_mads)} | Total: {len(top_entries)} entries ===")
    for e in top_entries:
        print(f'→ {e["Patient"]} | Slice {e["Slice"]} | Ch {e["Channel"]:02d} → MAD={e["MAD"]:.4f}, MSE={e["MSE"]:.4f}, SSIM={e["SSIM"]:.4f}')


In [ ]:
import re
from collections import defaultdict

# Load the raw text
with open("brightness_analysis_results_everything2.txt", "r") as file:
    lines = file.readlines()

# Storage for parsed entries
entries = []

current_slice = None
current_layer = None
current_path = None

# Regular expressions
path_re = re.compile(r"=== path:(.+)")
layer_re = re.compile(r"=== Layer: (.+) \| Channels: (\d+)")
line_re = re.compile(r"Slice (\d+) \| (.+) \| Ch (\d+) → MAD=(\d+\.\d+), MSE=(\d+\.\d+), SSIM=(\d+\.\d+)")

# Parse the file
for line in lines:
    path_match = path_re.match(line)
    if path_match:
        current_path = path_match.group(1).strip()
        continue

    layer_match = layer_re.match(line)
    if layer_match:
        current_layer = layer_match.group(1).strip()
        continue

    data_match = line_re.match(line)
    if data_match:
        slice_idx = int(data_match.group(1))
        layer_name = data_match.group(2).strip()
        ch = int(data_match.group(3))
        mad = float(data_match.group(4))
        mse = float(data_match.group(5))
        ssim = float(data_match.group(6))

        entries.append({
            "Patient": current_path,
            "Slice": slice_idx,
            "Layer": layer_name,
            "Channel": ch,
            "MAD": mad,
            "MSE": mse,
            "SSIM": ssim
        })

# Group by layer
layer_to_entries = defaultdict(list)
for e in entries:
    layer_to_entries[e["Layer"]].append(e)

# Save results to file and print
with open("top3_mad_per_layer.txt", "w") as out:
    out.write("Top 3 MAD entries per layer (including ties):\n")

    print("Top 3 MAD entries per layer (including ties):\n")

    for layer, items in layer_to_entries.items():
        sorted_items = sorted(items, key=lambda x: -x["MAD"])
        top_mads = sorted(set([x["MAD"] for x in sorted_items]), reverse=True)[:3]
        top_entries = [e for e in sorted_items if e["MAD"] in top_mads]

        header = f"\n=== Layer: {layer} | Top MADs: {', '.join(f'{m:.4f}' for m in top_mads)} | Total: {len(top_entries)} entries ==="
        print(header)
        out.write(header + "\n")

        for e in top_entries:
            line = f'→ {e["Patient"]} | Slice {e["Slice"]} | Ch {e["Channel"]:02d} → MAD={e["MAD"]:.4f}, MSE={e["MSE"]:.4f}, SSIM={e["SSIM"]:.4f}'
            print(line)
            out.write(line + "\n")


In [ ]:
import re
from collections import defaultdict, Counter

# Load the raw text file
with open("brightness_analysis_results_everything2.txt", "r") as file:
    lines = file.readlines()

# Prepare regex patterns
path_re = re.compile(r"=== path:(.+)")
layer_re = re.compile(r"=== Layer: (.+) \| Channels: (\d+)")
line_re = re.compile(r"Slice (\d+) \| (.+) \| Ch (\d+) → MAD=(\d+\.\d+), MSE=(\d+\.\d+), SSIM=(\d+\.\d+)")

entries = []
current_path = None
current_layer = None

# Parse file
for line in lines:
    path_match = path_re.match(line)
    if path_match:
        current_path = path_match.group(1).strip()
        continue

    layer_match = layer_re.match(line)
    if layer_match:
        current_layer = layer_match.group(1).strip()
        continue

    line_match = line_re.match(line)
    if line_match:
        slice_id = int(line_match.group(1))
        layer = line_match.group(2).strip()
        channel = int(line_match.group(3))
        mad = float(line_match.group(4))

        entries.append({
            "Layer": layer,
            "Slice": slice_id,
            "Channel": channel,
            "MAD": mad
        })

# Organize entries per layer
layer_dict = defaultdict(list)
for e in entries:
    layer_dict[e["Layer"]].append(e)

# Analyze per layer
print("🔍 Most common Channels and Slices in top MAD entries per Layer:\n")

for layer, layer_entries in layer_dict.items():
    # Get top 3 MADs per layer (including ties)
    sorted_entries = sorted(layer_entries, key=lambda x: -x["MAD"])
    top_mads = sorted(set([e["MAD"] for e in sorted_entries]), reverse=True)[:3]
    top_entries = [e for e in sorted_entries if e["MAD"] in top_mads]

    ch_counter = Counter([e["Channel"] for e in top_entries])
    sl_counter = Counter([e["Slice"] for e in top_entries])

    print(f"\n=== Layer: {layer} | Top MADs: {', '.join(f'{m:.4f}' for m in top_mads)} | Total: {len(top_entries)} entries ===")
    print("Top Channels: ", ch_counter.most_common(5))
    print("Top Slices:   ", sl_counter.most_common(5))


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Kanal verisi (Layer -> List of (Channel, Count))
layer_to_top_channels = {
    '0.conv.unit0.adn.N': [(2, 20), (3, 20)],
    '0.conv.unit1.adn.N': [(1, 20), (6, 10)],
    '0.conv.unit2.adn.N': [(6, 30)],
    '0.conv.unit3.adn.N': [(2, 30)],
    '1.submodule.0.conv.unit0.adn.N': [(1, 10), (15, 10), (5, 10)],
    '1.submodule.0.conv.unit1.adn.N': [(10, 20), (5, 10)],
    '1.submodule.0.conv.unit2.adn.N': [(10, 20), (12, 10)],
    '1.submodule.0.conv.unit3.adn.N': [(8, 30)],
    '1.submodule.1.submodule.0.conv.unit0.adn.N': [(1, 10), (31, 10), (14, 10)],
    '1.submodule.1.submodule.0.conv.unit1.adn.N': [(17, 20), (23, 10)],
    '1.submodule.1.submodule.0.conv.unit2.adn.N': [(29, 10), (31, 10), (25, 10)],
    '1.submodule.1.submodule.0.conv.unit3.adn.N': [(30, 30)],
    '1.submodule.1.submodule.1.submodule.0.conv.unit0.adn.N': [(1, 30)],
    '1.submodule.1.submodule.1.submodule.0.conv.unit1.adn.N': [(14, 10), (48, 10), (56, 10)],
    '1.submodule.1.submodule.1.submodule.0.conv.unit2.adn.N': [(27, 20), (13, 10)],
    '1.submodule.1.submodule.1.submodule.0.conv.unit3.adn.N': [(42, 20), (38, 10)],
    '1.submodule.1.submodule.2.0.adn.N': [(5, 20), (7, 10)],
    '1.submodule.1.submodule.2.1.conv.unit0.adn.N': [(4, 20), (14, 10)],
    '1.submodule.2.0.adn.N': [(3, 20), (2, 10)],
    '1.submodule.2.1.conv.unit0.adn.N': [(5, 20), (6, 10)],
    '2.0.adn.N': [(1, 20), (0, 10)],
}

# Veriyi düzleştir
bars = []
labels = []

for layer, tuples in layer_to_top_channels.items():
    for (channel, count) in tuples:
        bars.append(count)
        labels.append(f"{layer}\nCh {channel}")

# Grafik çizimi
fig, ax = plt.subplots(figsize=(14, 10))
bars_plot = ax.barh(range(len(bars)), bars, color='lightcoral')

# Etiketleme
ax.set_yticks(range(len(bars)))
ax.set_yticklabels(labels, fontsize=8)
ax.invert_yaxis()
ax.set_xlabel("Frequency in Top MAD Entries")
ax.set_title("Top Channel Frequencies per Layer", fontsize=14)
ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

# Her bar'ın sonuna değeri yaz
for i, bar in enumerate(bars_plot):
    width = bar.get_width()
    ax.text(width + 0.5, bar.get_y() + bar.get_height()/2,
            str(bars[i]), va='center', fontsize=8)

plt.tight_layout()
plt.savefig("improved_top_channels_per_layer.png", dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Veriyi yeniden tanımlayalım
layer_to_top_channels = {
    '0.conv.unit0.adn.N': [(2, 20), (3, 20)],
    '0.conv.unit1.adn.N': [(1, 20), (6, 10)],
    '0.conv.unit2.adn.N': [(6, 30)],
    '0.conv.unit3.adn.N': [(2, 30)],
    '1.submodule.0.conv.unit0.adn.N': [(1, 10), (15, 10), (5, 10)],
    '1.submodule.0.conv.unit1.adn.N': [(10, 20), (5, 10)],
    '1.submodule.0.conv.unit2.adn.N': [(10, 20), (12, 10)],
    '1.submodule.0.conv.unit3.adn.N': [(8, 30)],
    '1.submodule.1.submodule.0.conv.unit0.adn.N': [(1, 10), (31, 10), (14, 10)],
    '1.submodule.1.submodule.0.conv.unit1.adn.N': [(17, 20), (23, 10)],
    '1.submodule.1.submodule.0.conv.unit2.adn.N': [(29, 10), (31, 10), (25, 10)],
    '1.submodule.1.submodule.0.conv.unit3.adn.N': [(30, 30)],
    '1.submodule.1.submodule.1.submodule.0.conv.unit0.adn.N': [(1, 30)],
    '1.submodule.1.submodule.1.submodule.0.conv.unit1.adn.N': [(14, 10), (48, 10), (56, 10)],
    '1.submodule.1.submodule.1.submodule.0.conv.unit2.adn.N': [(27, 20), (13, 10)],
    '1.submodule.1.submodule.1.submodule.0.conv.unit3.adn.N': [(42, 20), (38, 10)],
    '1.submodule.1.submodule.2.0.adn.N': [(5, 20), (7, 10)],
    '1.submodule.1.submodule.2.1.conv.unit0.adn.N': [(4, 20), (14, 10)],
    '1.submodule.2.0.adn.N': [(3, 20), (2, 10)],
    '1.submodule.2.1.conv.unit0.adn.N': [(5, 20), (6, 10)],
    '2.0.adn.N': [(1, 20), (0, 10)],
}

# Veriyi düzleştir
bars = []
labels = []

# Daha okunabilir kısa layer isimleri oluştur
def shorten_layer(layer):
    return layer.split(".")[0] + "." + layer.split(".")[1] + (f"...{layer[-10:]}" if len(layer) > 40 else "")

for layer, tuples in layer_to_top_channels.items():
    for (channel, count) in tuples:
        bars.append(count)
        short_layer = shorten_layer(layer)
        labels.append(f"{short_layer}\nCh{channel}")

# Çizim
fig, ax = plt.subplots(figsize=(10, 14))
bars_plot = ax.barh(range(len(bars)), bars, color='cornflowerblue')

# Etiketler
ax.set_yticks(range(len(bars)))
ax.set_yticklabels(labels, fontsize=8)
ax.invert_yaxis()
ax.set_xlabel("Frequency in Top MAD Entries", fontsize=12)
ax.set_title("Top Channel Frequencies per Layer", fontsize=14)
ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

# Bar'ların sonuna değer yaz
for i, bar in enumerate(bars_plot):
    width = bar.get_width()
    ax.text(width + 0.3, bar.get_y() + bar.get_height()/2,
            str(bars[i]), va='center', fontsize=8)

plt.tight_layout()
plt.savefig("top_channel_frequencies_readable.png", dpi=300)
plt.show()


In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

# Load and parse the file
entries = []
with open("brightness_analysis_results_everything2.txt", "r") as file:
    lines = file.readlines()

path_re = re.compile(r"=== path:(.+)")
layer_re = re.compile(r"=== Layer: (.+) \| Channels: (\d+)")
line_re = re.compile(r"Slice (\d+) \| (.+) \| Ch (\d+) → MAD=(\d+\.\d+), MSE=(\d+\.\d+), SSIM=(\d+\.\d+)")

current_path = None
current_layer = None

for line in lines:
    if line.startswith("=== path:"):
        path_match = path_re.match(line)
        if path_match:
            current_path = path_match.group(1).strip()
        continue

    layer_match = layer_re.match(line)
    if layer_match:
        current_layer = layer_match.group(1).strip()
        continue

    data_match = line_re.match(line)
    if data_match:
        slice_idx = int(data_match.group(1))
        ch = int(data_match.group(3))
        mad = float(data_match.group(4))

        entries.append({
            "Layer": current_layer,
            "Channel": ch,
            "MAD": mad
        })

# Create DataFrame
df = pd.DataFrame(entries)

# Pivot table: average MAD per (Layer, Channel)
heatmap_df = df.groupby(["Layer", "Channel"])["MAD"].mean().unstack(fill_value=0)

# Plot heatmap
plt.figure(figsize=(18, 10))
sns.heatmap(heatmap_df, cmap="magma", linewidths=0.5, linecolor='gray')
plt.title("🔆 Mean MAD per Layer and Channel (Brightness Sensitivity)", fontsize=16)
plt.xlabel("Channel", fontsize=12)
plt.ylabel("Layer", fontsize=12)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np

def dice_score(pred_mask, true_mask, class_id=3):
    pred_bin = (pred_mask == class_id).astype(np.uint8)
    true_bin = (true_mask == class_id).astype(np.uint8)
    intersection = np.sum(pred_bin * true_bin)
    total = np.sum(pred_bin) + np.sum(true_bin)
    return 1.0 if total == 0 else 2.0 * intersection / total

for path in patient_ids:
    img = all_images[path]           # shape: (slices, H, W)
    gt = all_labels[path]           # shape: (slices, H, W)

    bright_img = img.copy()
    indices = np.where(gt == 3)
    bright_img[indices] *= 0.1

    pred_bright = model.predict(bright_img)         # shape: (slices, classes, H, W)
    seg_bright = np.argmax(pred_bright, axis=1)     # shape: (slices, H, W)

    num_slices = bright_img.shape[0]
    cols = 3
    rows = num_slices

    fig, axs = plt.subplots(rows, cols, figsize=(cols * 3.2, rows * 3.2))
    if rows == 1:
        axs = np.expand_dims(axs, axis=0)

    for i in range(num_slices):
        gt_slice = gt[i]
        pred_slice = seg_bright[i]
        score = dice_score(pred_slice, gt_slice, class_id=3)

        axs[i, 0].set_ylabel(f"{path}\nSlice {i}\nDice: {score:.4f}", fontsize=8, rotation=0, labelpad=80, va='center')

        axs[i, 0].imshow(bright_img[i], cmap="gray")
        axs[i, 0].set_title("Original", fontsize=8)
        axs[i, 0].axis("off")

        axs[i, 1].imshow(bright_img[i], cmap="gray")
        axs[i, 1].imshow(gt[i], alpha=0.4)
        axs[i, 1].set_title("Ground Truth", fontsize=8)
        axs[i, 1].axis("off")

        axs[i, 2].imshow(bright_img[i], cmap="gray")
        axs[i, 2].imshow(pred_slice, alpha=0.4)
        axs[i, 2].set_title("Segmentation", fontsize=8)
        axs[i, 2].axis("off")

    plt.tight_layout()
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def dice_score(pred_mask, true_mask, class_id=3):
    pred_bin = (pred_mask == class_id).astype(np.uint8)
    true_bin = (true_mask == class_id).astype(np.uint8)
    intersection = np.sum(pred_bin * true_bin)
    total = np.sum(pred_bin) + np.sum(true_bin)
    return 1.0 if total == 0 else 2.0 * intersection / total

for path in patient_ids:
    img = all_images[path]           # (slices, H, W)
    gt = all_labels[path]           # (slices, H, W)

    bright_img = img.copy()
    indices = np.where(gt == 3)
    bright_img[indices] *= 0.1

    pred_bright = model.predict(bright_img)         # (slices, classes, H, W)
    seg_bright = np.argmax(pred_bright, axis=1)     # (slices, H, W)

    num_slices = bright_img.shape[0]

    for i in range(num_slices):
        gt_slice = gt[i]
        pred_slice = seg_bright[i]
        score = dice_score(pred_slice, gt_slice, class_id=3)

        print(f"▶ {path} – Slice {i} – Dice: {score:.4f}")  # ✅ This line is key

        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(bright_img[i], cmap="gray")
        plt.title("Brightness-Adjusted Input")
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.imshow(bright_img[i], cmap="gray")
        plt.imshow(gt[i], alpha=0.4)
        plt.title("GT Overlay")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.imshow(bright_img[i], cmap="gray")
        plt.imshow(pred_slice, alpha=0.4)
        plt.title("Prediction on Adjusted")
        plt.axis("off")

        plt.tight_layout()
        plt.show()
